### 00_file_audit_and_overview — Columns-only Explainer

### What this step does
Scans `data_dir` **recursively** and builds a lightweight inventory of **column names only** for each data file:
- Tabular: `.csv`, `.xlsx`, `.xls`, `.parquet`
- Geospatial: `.shp` (Shapefile), `.gpkg` (GeoPackage)

It also expands **all Excel sheets** and **all GeoPackage layers**, so each sheet/layer gets its own entry.

### Why this is useful
- Fast schema check without loading full datasets
- Spot mismatched/duplicate schemas across folders
- Create a shareable “columns dictionary” for collaborators

### Inputs
Set your root folder:
```python
data_dir = "/Users/rosstaylor/.../REACH-Map-NHS-SW/data/raw"


In [1]:
# ------------------------------------------------------------------
# Step 0 – Purpose
# ------------------------------------------------------------------
# Inspect all files in a given folder to:
# - Identify file types
# - Attempt to load them with pandas
# - Print file name, shape, column names, and header preview

In [9]:
# ------------------------------------------------------------------
# Step 1 – Imports
# ------------------------------------------------------------------

import pandas as pd
import geopandas as gpd
import fiona
from pathlib import Path
import os, json, math, time
from pathlib import Path
import pandas as pd


In [3]:
# ------------------------------------------------------------------
# Step 2 – Define Root Data Directory
# ------------------------------------------------------------------
data_dir = "/Users/rosstaylor/Downloads/Code Repositories/REACH Map (NHS SW)/GitHub Repo/REACH-Map-NHS-SW/data/raw"


In [8]:
# ------------------------------------------------------------------
# Step 3 – Walk sub-folders from data_dir and inspect files
# ------------------------------------------------------------------

TARGET_EXTS = {".csv", ".xlsx", ".xls", ".parquet", ".shp", ".gpkg"}
IGNORE_DIRS = {"processed", ".git", "__pycache__"}

summary = []

for root, dirs, files in os.walk(data_dir):
    # prune ignored directories in-place
    dirs[:] = [d for d in dirs if d not in IGNORE_DIRS and not d.startswith(".")]

    for filename in files:
        ext = os.path.splitext(filename)[-1].lower()
        if ext not in TARGET_EXTS:
            continue

        filepath = os.path.join(root, filename)

        # inspect (your Step 2 function)
        columns, n_columns, error = inspect_file(filepath)

        # file size may fail on odd perms; keep scanning
        try:
            size_mb = round(os.path.getsize(filepath) / (1024 * 1024), 2)
        except OSError as e:
            size_mb = None
            error = f"{error} | size_error: {e}" if error else f"size_error: {e}"

        summary.append({
            "relative_path": os.path.relpath(filepath, data_dir),
            "filename": filename,
            "filetype": ext,
            "size_MB": size_mb,
            "n_columns": n_columns,
            "columns": (columns[:25] if columns else None),  # truncate long lists for readability
            "error": error
        })

summary_df = pd.DataFrame(summary).sort_values(["relative_path", "filetype"], na_position="last")

# quick view
print(summary_df[["relative_path", "filetype", "n_columns", "size_MB", "error"]].to_string(index=False))

# optional: save an inventory next to data/raw
# out_csv = Path(data_dir).parent / "data_inventory.csv"
# summary_df.to_csv(out_csv, index=False)
# print(f"\nSaved: {out_csv}")

                                                           relative_path filetype  n_columns  size_MB error
       demographics/Rural_Urban_Classification_(2021)_of_LSOAs_in_EW.csv     .csv          7     2.87  None
                       demographics/census_2021_general_health_lsoas.csv     .csv          7     0.11  None
                                          demographics/lsoa_imd_2019.csv     .csv          2     0.06  None
                                       gkpg/LSOA_5-year_age_segment.gpkg    .gpkg         32    68.66  None
                                             gkpg/LSOA_discrete_age.gpkg    .gpkg        108    70.82  None
             health_infrastructure/NHS_SW_Acute_Hospitals_Capability.csv     .csv         23     0.01  None
health_infrastructure/NHS_SW_Community_Diagnostic_Centres_Capability.csv     .csv         22     0.01  None
         health_infrastructure/NHS_SW_Community_Hospitals_Capability.csv     .csv         20     0.02  None
                            

In [14]:
# ------------------------------------------------------------------
# Step 4 – Columns-only inventory
# ------------------------------------------------------------------

TARGET_EXTS = {'.csv', '.xlsx', '.xls', '.parquet', '.shp', '.gpkg'}

def columns_only(filepath):
    """Return a list of {layer_or_sheet, columns} for the file."""
    ext = Path(filepath).suffix.lower()
    rows = []

    if ext == '.csv':
        df = pd.read_csv(filepath, nrows=0)  # header only
        rows.append(dict(layer_or_sheet=None, columns=list(df.columns)))

    elif ext in ('.xlsx', '.xls'):
        xls = pd.ExcelFile(filepath)
        for sh in xls.sheet_names:
            df = pd.read_excel(xls, sheet_name=sh, nrows=0)
            rows.append(dict(layer_or_sheet=sh, columns=list(df.columns)))

    elif ext == '.parquet':
        try:
            import pyarrow.parquet as pq
            cols = pq.ParquetFile(filepath).schema.names
        except Exception:
            cols = list(pd.read_parquet(filepath, columns=None).columns)
        rows.append(dict(layer_or_sheet=None, columns=list(cols)))

    elif ext == '.shp':
        import fiona
        with fiona.open(filepath) as src:
            cols = list(src.schema.get('properties', {}).keys())
        rows.append(dict(layer_or_sheet=None, columns=cols))

    elif ext == '.gpkg':
        import fiona
        for lyr in fiona.listlayers(filepath):
            with fiona.open(filepath, layer=lyr) as src:
                cols = list(src.schema.get('properties', {}).keys())
            rows.append(dict(layer_or_sheet=lyr, columns=cols))

    return rows


# Walk tree and collect columns only
records = []
for root, _, files in os.walk(data_dir):
    for name in files:
        ext = Path(name).suffix.lower()
        if ext not in TARGET_EXTS:
            continue
        path = os.path.join(root, name)
        rel = os.path.relpath(path, data_dir)

        try:
            for rec in columns_only(path):
                records.append({
                    'relative_path': rel,
                    'filetype': ext,
                    'layer_or_sheet': rec['layer_or_sheet'],
                    'columns': rec['columns'],
                })
        except Exception as e:
            records.append({
                'relative_path': rel,
                'filetype': ext,
                'layer_or_sheet': None,
                'columns': None,
                'error': str(e),
            })

cols_df = pd.DataFrame.from_records(records).sort_values(['relative_path','layer_or_sheet'])

# Quick view
print(cols_df.to_string(index=False))

# Optional saves:
# cols_df.to_csv('columns_only.csv', index=False)
# cols_df.to_json('columns_only.json', orient='records', indent=2)


                                                           relative_path filetype             layer_or_sheet                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 columns
       demographics/Rural_Urban_Classification_(2021)_of_LSOAs_in_EW.csv     .csv                       None                                                                                                                                                                                                                                                                                                       